In [59]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.functions import to_date, col, explode
from pyspark.sql.types import *
from pyspark.sql.functions import collect_list

In [ ]:
import os, pathlib, shutil
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"]       = r"C:\hadoop\bin;" + os.environ["PATH"]
os.environ["AWS_PROFILE"] = "ryanj"
os.environ["AWS_REGION"]  = "us-west-2"
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"] = rf'{os.environ["HADOOP_HOME"]}\bin;' + os.environ["PATH"]
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.6 "
    "--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem "
    "--conf spark.hadoop.fs.s3a.aws.region=us-west-2 "
    "--conf spark.hadoop.fs.s3a.aws.credentials.provider=com.amazonaws.auth.DefaultAWSCredentialsProviderChain "
    "--conf spark.hadoop.io.native.lib.available=false "
    "--conf spark.hadoop.fs.s3a.fast.upload=true "
    "--conf spark.hadoop.fs.s3a.fast.upload.buffer=bytebuffer "
    "pyspark-shell"
)

In [3]:
spark = (SparkSession.builder
         .appName("sumo-s3a")
         # make sure S3A is wired up; you can keep these if they helped earlier
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
         .config("spark.hadoop.fs.s3a.aws.region", "us-west-2")
         .getOrCreate())

C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
hc = spark.sparkContext._jsc.hadoopConfiguration()
print("S3A impl:", hc.get("fs.s3a.impl")) 

S3A impl: org.apache.hadoop.fs.s3a.S3AFileSystem


In [5]:
# make a writable local temp dir
spark_local = r"C:\tmp\spark"
pathlib.Path(spark_local).mkdir(parents=True, exist_ok=True)

In [6]:
print("Spark:", spark.version)
print("Hadoop:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

Spark: 4.0.1
Hadoop: 3.4.1


In [7]:
spark.conf.set("spark.hadoop.fs.s3a.connection.establish.timeout", "30000")   # 30s
spark.conf.set("spark.hadoop.fs.s3a.threads.keepalivetime",        "60000")   # 60s
spark.conf.set("spark.hadoop.fs.s3a.connection.ttl",              "300000")   # 5m
spark.conf.set("spark.hadoop.fs.s3a.retry.interval",                 "500")   # 500ms
spark.conf.set("spark.hadoop.fs.s3a.retry.throttle.interval",        "100")   # 100ms

In [8]:
# convert human-friendly durations to plain integers (milliseconds)
hconf = spark.sparkContext._jsc.hadoopConfiguration()

fixes = {
    "fs.s3a.connection.establish.timeout": "30000",    # 30s
    "fs.s3a.threads.keepalivetime": "60000",           # 60s
    "fs.s3a.retry.interval": "500",                    # 500ms
    "fs.s3a.retry.throttle.interval": "100",           # 100ms
    "fs.s3a.connection.ttl": "300000",                 # 5m
    "fs.s3a.assumed.role.session.duration": "1800000"  # 30m
}
for k, v in fixes.items():
    hconf.set(k, v)

# (optional) double-check one
print("keepalive:", hconf.get("fs.s3a.threads.keepalivetime"))


keepalive: 60000


In [9]:
import re

hconf = spark.sparkContext._jsc.hadoopConfiguration()

# report any time-looking values with units
offenders = []
it = hconf.iterator()
pairs = []
while it.hasNext():
    e = it.next()
    pairs.append((e.getKey(), e.getValue()))

r_time = re.compile(r"^\s*\d+\s*(ms|s|m|h|d)\s*$", re.I)

for k, v in pairs:
    if r_time.match(str(v)):
        offenders.append((k, v))

print("Offenders before fix:")
for k, v in offenders:
    print(f"  {k} = {v}")

# convert unit-suffixed durations to milliseconds
UNIT_MS = {"ms":1, "s":1000, "m":60_000, "h":3_600_000, "d":86_400_000}

def to_ms_str(val):
    m = re.fullmatch(r"\s*(\d+)\s*(ms|s|m|h|d)\s*", str(val).lower())
    if not m: 
        return None
    n = int(m.group(1)); u = m.group(2)
    return str(n * UNIT_MS[u])

changed = []
for k, v in pairs:
    ms = to_ms_str(v)
    if ms and ms != v:
        hconf.set(k, ms)
        changed.append((k, v, ms))

print("\nNormalized to milliseconds:")
for k, old, new in changed:
    print(f"  {k}: {old} -> {new}")

# show any stragglers still using units
left = []
it = hconf.iterator()
while it.hasNext():
    e = it.next()
    if r_time.match(str(e.getValue())):
        left.append((e.getKey(), e.getValue()))
print("\nRemaining with units (should be empty):", left)


Offenders before fix:
  fs.s3a.vectored.read.max.merged.size = 2M
  yarn.router.subcluster.cleaner.interval.time = 60s
  yarn.resourcemanager.delegation.token.remove-scan-interval = 1h
  yarn.router.interceptor.user-thread-pool.keep-alive-time = 30s
  fs.s3a.multipart.purge.age = 24h
  yarn.federation.state-store.heartbeat.initial-delay = 30s
  hadoop.security.groups.shell.command.timeout = 0s
  hadoop.service.shutdown.timeout = 30s
  yarn.federation.state-store.clean-up-retry-sleep-time = 1s
  fs.s3a.multipart.threshold = 128M
  yarn.router.subcluster.heartbeat.expiration.time = 30m
  yarn.resourcemanager.delegation-token-renewer.thread-retry-interval = 60s
  fs.s3a.block.size = 32M
  yarn.router.submit.interval.time = 10ms
  yarn.federation.gpg.webapp.connect-timeout = 30s
  yarn.federation.state-store.sql.max-life-time = 30m
  fs.s3a.multipart.size = 64M
  yarn.federation.state-store.sql.conn-time-out = 10s
  fs.azure.sas.expiry.period = 90d
  yarn.resourcemanager.delegation-token-r

In [10]:
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")  # avoids choking on stray files

# bronze reads and silver writes

In [10]:


df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishi_matches/"))


In [11]:

# After your JSON read
df.printSchema()
# You'll likely see: root
#  |-- 0: array (nullable = true)
#  |    |-- element: struct (contains your fields)

rows = df.select(explode(col("records")).alias("match"))
flat = rows.select("match.*")   # expand struct into columns
flat.show(20, truncate=False)
flat.printSchema()

root
 |-- limit: long (nullable = true)
 |-- records: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bashoId: string (nullable = true)
 |    |    |-- day: long (nullable = true)
 |    |    |-- division: string (nullable = true)
 |    |    |-- eastId: long (nullable = true)
 |    |    |-- eastRank: string (nullable = true)
 |    |    |-- eastShikona: string (nullable = true)
 |    |    |-- kimarite: string (nullable = true)
 |    |    |-- matchNo: long (nullable = true)
 |    |    |-- westId: long (nullable = true)
 |    |    |-- westRank: string (nullable = true)
 |    |    |-- westShikona: string (nullable = true)
 |    |    |-- winnerEn: string (nullable = true)
 |    |    |-- winnerId: long (nullable = true)
 |    |    |-- winnerJp: string (nullable = true)
 |-- skip: long (nullable = true)
 |-- total: long (nullable = true)

+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-

convert to parquet

In [12]:
silver = (flat
          .dropDuplicates())          # typical dedupe key

In [13]:
silver = (silver
         .withColumn("year", F.col("bashoId").cast("string").substr(1,4))
          .withColumn("month", F.col("bashoId").cast("string").substr(5,2))
          .withColumn("match_date",
                     F.to_date(
                         F.concat_ws("-", F.col("year"), F.col("month"), F.lpad(F.col("day"), 2, "0")),
                             "yyyy-MM-dd"
                         )
                     )
          .withColumn(
              "match_id",
              F.concat_ws("", F.date_format(F.col("match_date"), "yyyyMMdd"), F.col("matchNo").cast("string")).cast("long")
          )
          .drop("year", "month")
         )

In [14]:
silver.show(10)

+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
|bashoId|day|division|eastId|          eastRank|eastShikona|   kimarite|matchNo|westId|          westRank|westShikona|   winnerEn|winnerId|winnerJp|match_date|  match_id|
+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
| 201501|  1|Makuuchi|  3835| Maegashira 7 East| Kyokutenho|   yorikiri|     10|   670| Maegashira 7 West| Chiyootori| Kyokutenho|    3835|        |2015-01-01|2015010110|
| 201407| 15|Makuuchi|  3835|Maegashira 12 East| Kyokutenho|   yorikiri|      2|  4126|Maegashira 15 West|  Tokitenku|  Tokitenku|    4126|        |2014-07-15| 201407152|
| 201309| 10|Makuuchi|  4177|Maegashira 13 West|    Homasho|   yorikiri|     10|  3835| Maegashira 6 West| Kyokutenho|    Homasho|    4177|      

In [15]:
# optional sanity checks:
hc = spark.sparkContext._jsc.hadoopConfiguration()
print("io.native.lib.available =", hc.get("io.native.lib.available"))
print("fs.s3a.fast.upload =", hc.get("fs.s3a.fast.upload"))
print("fs.s3a.fast.upload.buffer =", hc.get("fs.s3a.fast.upload.buffer"))

io.native.lib.available = false
fs.s3a.fast.upload = true
fs.s3a.fast.upload.buffer = bytebuffer


In [16]:
(silver.write
 .mode("overwrite")
 .option("compression", "snappy")
 .partitionBy("bashoId")                        # or by year/month/day
 .parquet("s3a://ryans-sumo-bucket/silver/rikishi_matches/"))

In [28]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishis/"))

In [30]:
df.show(10)

+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+
|           birthDate|      currentRank| debut|height|       heya|  id|               intai|nskId|           shikonaEn|             shikonaJp|            shusshin|sumodbId|weight|
+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+
|1975-08-22T00:00:00Z|             NULL|199103| 187.5|   Kasugano|3739|2011-01-01T00:00:00Z|    0|Kasuganishiki Tak...|                  NULL|Chiba-ken, Isumi-...|     107| 146.3|
|2000-10-21T00:00:00Z|   Jonidan 3 East|201903| 176.0|Shikoroyama| 279|2025-03-26T00:00:00Z| 3959|           Kawabuchi|        川渕(かわぶち)|Ishikawa-ken, Kan...|   12568| 118.0|
|1977-04-15T00:00:00Z| Jonidan 100 West|199601| 170.3|  Isegahama| 460|2025-05-28T00:00:00Z| 1407|        

In [31]:
(df.write
 .mode("overwrite")
 .option("compression", "snappy")
 .parquet("s3a://ryans-sumo-bucket/silver/rikishis/"))

In [32]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishi_stats/"))

df = df.withColumn(
    "rikishi_id",
    F.regexp_extract(F.input_file_name(), r"rikishi_(\d+)\.json", 1)
)

(df.write
 .mode("overwrite")
 .option("compression", "snappy")
 .parquet("s3a://ryans-sumo-bucket/silver/rikishi_stats/"))

In [33]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/basho/"))
(df.write
 .mode("overwrite")
 .option("compression", "snappy")
 .partitionBy("location")
 .parquet("s3a://ryans-sumo-bucket/silver/bashos/"))

# silvers have been written

In [11]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishi_matches/"))

In [12]:
df.show(10)

+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
|day| division|eastId|          eastRank|   eastShikona|   kimarite|matchNo|westId|          westRank|westShikona|   winnerEn|winnerId|winnerJp|match_date|  match_id|
+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
|  8| Makuuchi|   794| Maegashira 3 East|         Terao|   yorikiri|     14|  4071|   Sekiwake 1 West|       Kaio|       Kaio|    4071|        |1996-05-08|1996050814|
|  4|Makushita|  3827| Makushita 17 East|   Chiyotenzan|shitatenage|     24|  3837| Makushita 15 East| Kitazakura|Chiyotenzan|    3827|        |1996-05-04|1996050424|
|  3|    Juryo|  3835|      Juryo 7 West|    Kyokutenho|   yorikiri|     12|  5078|      Juryo 1 West|Tokitsunada| Kyokutenho|    3835|        |1996-05-03|1996050312

In [13]:
df.count()

738480

In [104]:
matches = (df
      .withColumn(
        "westWin",
        F.when(F.col("winnerId") == F.col("westId"), F.lit(1)).otherwise(F.lit(0))
        )
     )

In [15]:
rikishis = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishis/"))

rikishi_stats = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishi_stats/"))


bashos = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/bashos/"))

In [16]:
matches.show(10)

+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+-------+
|day| division|eastId|          eastRank|   eastShikona|   kimarite|matchNo|westId|          westRank|westShikona|   winnerEn|winnerId|winnerJp|match_date|  match_id|westWin|
+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+-------+
|  8| Makuuchi|   794| Maegashira 3 East|         Terao|   yorikiri|     14|  4071|   Sekiwake 1 West|       Kaio|       Kaio|    4071|        |1996-05-08|1996050814|      1|
|  4|Makushita|  3827| Makushita 17 East|   Chiyotenzan|shitatenage|     24|  3837| Makushita 15 East| Kitazakura|Chiyotenzan|    3827|        |1996-05-04|1996050424|      0|
|  3|    Juryo|  3835|      Juryo 7 West|    Kyokutenho|   yorikiri|     12|  5078|      Juryo 1 West|Tokitsunada| Kyokutenho

In [17]:
rikishis.show(10)

+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+
|           birthDate|      currentRank| debut|height|       heya|  id|               intai|nskId|           shikonaEn|             shikonaJp|            shusshin|sumodbId|weight|
+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+
|1975-08-22T00:00:00Z|             NULL|199103| 187.5|   Kasugano|3739|2011-01-01T00:00:00Z|    0|Kasuganishiki Tak...|                  NULL|Chiba-ken, Isumi-...|     107| 146.3|
|2000-10-21T00:00:00Z|   Jonidan 3 East|201903| 176.0|Shikoroyama| 279|2025-03-26T00:00:00Z| 3959|           Kawabuchi|        川渕(かわぶち)|Ishikawa-ken, Kan...|   12568| 118.0|
|1977-04-15T00:00:00Z| Jonidan 100 West|199601| 170.3|  Isegahama| 460|2025-05-28T00:00:00Z| 1407|        

In [20]:
bashos.printSchema()

root
 |-- date: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- specialPrizes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- rikishiId: long (nullable = true)
 |    |    |-- shikonaEn: string (nullable = true)
 |    |    |-- shikonaJp: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- startDate: string (nullable = true)
 |-- yusho: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- rikishiId: long (nullable = true)
 |    |    |-- shikonaEn: string (nullable = true)
 |    |    |-- shikonaJp: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [21]:
# Rename `date` → `bashoId`, and keep start/end date
base = bashos.select(
    col("date").alias("bashoId"),
    col("startDate"),
    col("endDate")
)

yusho_flat = (
    bashos.withColumn("y", explode(col("yusho")))
      .select(
          col("date").alias("bashoId"),
          col("y.type").alias("yusho_type"),
          col("y.rikishiId").alias("rikishiId")
      )
)

# Pivot so each yusho_type becomes its own column
yusho_pivot = (
    yusho_flat.groupBy("bashoId")
              .pivot("yusho_type")
              .agg(collect_list("rikishiId"))
              .withColumnRenamed("Makuuchi", "Makuuchi_yusho")
              .withColumnRenamed("Juryo", "Juryo_yusho")
)
prizes_flat = (
    bashos.withColumn("p", explode(col("specialPrizes")))
      .select(
          col("date").alias("bashoId"),
          col("p.type").alias("prize_type"),
          col("p.rikishiId").alias("rikishiId")
      )
)


prizes_pivot = (
    prizes_flat.groupBy("bashoId")
               .pivot("prize_type")
               .agg(collect_list("rikishiId"))   # array of rikishiIds
               .withColumnRenamed("Shukun-sho", "Shukun_sho")
               .withColumnRenamed("Kanto-sho", "Kanto_sho")
               .withColumnRenamed("Gino-sho", "Gino_sho")
)
result = (base
          .join(yusho_pivot, on="bashoId", how="left")
          .join(prizes_pivot, on="bashoId", how="left"))

In [22]:
result.show(10)

+-------+--------------------+--------------------+-------+---------+-----------+---------+--------------+--------+------------+-------------+------------+
|bashoId|           startDate|             endDate|Jonidan|Jonokuchi|Juryo_yusho|Makushita|Makuuchi_yusho|Sandanme|    Gino_sho|    Kanto_sho|  Shukun_sho|
+-------+--------------------+--------------------+-------+---------+-----------+---------+--------------+--------+------------+-------------+------------+
| 202501|2025-01-12T00:00:00Z|2025-01-26T00:00:00Z|  [106]|    [159]|       [86]|     [91]|          [19]|  [8869]|        [41]|      [7, 50]|          []|
| 202305|2023-05-14T00:00:00Z|2023-05-28T00:00:00Z|  [609]|   [8862]|       [56]|    [110]|          [45]|   [219]|     [13, 7]|           []|        [38]|
| 201801|2018-01-14T00:00:00Z|2018-01-28T00:00:00Z|  [336]|     [81]|       [36]|     [12]|          [43]|    [69]|        [43]|     [15, 22]|        [43]|
| 201705|2017-05-14T00:00:00Z|2017-05-28T00:00:00Z|  [354]|     

In [23]:
result.printSchema()

root
 |-- bashoId: string (nullable = true)
 |-- startDate: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- Jonidan: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Jonokuchi: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Juryo_yusho: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Makushita: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Makuuchi_yusho: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Sandanme: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Gino_sho: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Kanto_sho: array (nullable = true)
 |    |-- element: long (containsNull = false)
 |-- Shukun_sho: array (nullable = true)
 |    |-- element: long (containsNull = false)



In [24]:
rikishi_stats.printSchema()

root
 |-- absenceByDivision: struct (nullable = true)
 |    |-- Jonidan: long (nullable = true)
 |    |-- Jonokuchi: long (nullable = true)
 |    |-- Juryo: long (nullable = true)
 |    |-- Makushita: long (nullable = true)
 |    |-- Makuuchi: long (nullable = true)
 |    |-- Sandanme: long (nullable = true)
 |-- basho: long (nullable = true)
 |-- bashoByDivision: struct (nullable = true)
 |    |-- Jonidan: long (nullable = true)
 |    |-- Jonokuchi: long (nullable = true)
 |    |-- Juryo: long (nullable = true)
 |    |-- Makushita: long (nullable = true)
 |    |-- Makuuchi: long (nullable = true)
 |    |-- Sandanme: long (nullable = true)
 |-- lossByDivision: struct (nullable = true)
 |    |-- Jonidan: long (nullable = true)
 |    |-- Jonokuchi: long (nullable = true)
 |    |-- Juryo: long (nullable = true)
 |    |-- Makushita: long (nullable = true)
 |    |-- Makuuchi: long (nullable = true)
 |    |-- Sandanme: long (nullable = true)
 |-- sansho: struct (nullable = true)
 |    |-- Gi

In [25]:
rikishi_stats = rikishi_stats.select(
    col("rikishi_id"),
    col("yusho"),
    col("yushoByDivision.Makuuchi").alias("makuuchi_yusho"),
    col("totalWins"),
    col("totalLosses"),
    col("totalMatches"),
    col("winsByDivision.Makuuchi").alias("makuuchiWins"),
    col("basho"),
    col("bashoByDivision.Makuuchi").alias("Makuuchi_basho"),
    col("sansho.Gino-sho").alias("Gino_sho"),
    col("sansho.Kanto-sho").alias("Kanto_sho"),
    col("sansho.Shukun-sho").alias("Shukun_sho")
)

In [26]:
rikishi_stats.show(10)

+----------+-----+--------------+---------+-----------+------------+------------+-----+--------------+--------+---------+----------+
|rikishi_id|yusho|makuuchi_yusho|totalWins|totalLosses|totalMatches|makuuchiWins|basho|Makuuchi_basho|Gino_sho|Kanto_sho|Shukun_sho|
+----------+-----+--------------+---------+-----------+------------+------------+-----+--------------+--------+---------+----------+
|        35|    3|          NULL|      753|        765|        1518|         405|  132|            58|    NULL|        3|      NULL|
|        44|    1|          NULL|      787|        593|        1380|         627|  120|            84|       3|        6|         4|
|      5306|    4|          NULL|      457|        433|         890|         164|  100|            26|    NULL|        1|      NULL|
|        16|    2|          NULL|      635|        648|        1283|         289|  114|            43|    NULL|        1|         1|
|       794|    2|          NULL|      858|        937|        1795| 

In [105]:
main = matches

In [31]:
rikishis = (rikishis
       .join(rikishi_stats, rikishis.id == rikishi_stats.rikishi_id, how = "left"))

In [32]:
rikishis.show(10)

+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+----------+-----+--------------+---------+-----------+------------+------------+-----+--------------+--------+---------+----------+
|           birthDate|      currentRank| debut|height|       heya|  id|               intai|nskId|           shikonaEn|             shikonaJp|            shusshin|sumodbId|weight|rikishi_id|yusho|makuuchi_yusho|totalWins|totalLosses|totalMatches|makuuchiWins|basho|Makuuchi_basho|Gino_sho|Kanto_sho|Shukun_sho|
+--------------------+-----------------+------+------+-----------+----+--------------------+-----+--------------------+----------------------+--------------------+--------+------+----------+-----+--------------+---------+-----------+------------+------------+-----+--------------+--------+---------+----------+
|1975-08-22T00:00:00Z|             NULL|199103| 187.5|   Kasugano|3

In [36]:
rikishis.printSchema()

root
 |-- birthDate: string (nullable = true)
 |-- currentRank: string (nullable = true)
 |-- debut: string (nullable = true)
 |-- height: double (nullable = true)
 |-- id: long (nullable = true)
 |-- intai: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- rikishi_id: string (nullable = true)
 |-- yusho: long (nullable = true)
 |-- makuuchi_yusho: long (nullable = true)
 |-- totalWins: long (nullable = true)
 |-- totalLosses: long (nullable = true)
 |-- totalMatches: long (nullable = true)
 |-- makuuchiWins: long (nullable = true)
 |-- basho: long (nullable = true)
 |-- Makuuchi_basho: long (nullable = true)
 |-- Gino_sho: long (nullable = true)
 |-- Kanto_sho: long (nullable = true)
 |-- Shukun_sho: long (nullable = true)



In [35]:
rikishis = (rikishis
           .drop("shikonaEn")
            .drop("shikonaJp")
            .drop("sumodbId")
            .drop("nskId")
            .drop("shusshin")
            .drop("heya")
           )

In [128]:
def add_prefix(df, prefix):
    return df.select([F.col(c).alias(f"{prefix}_{c}") for c in df.columns])

# Add prefixes
west = add_prefix(rikishis, "west")
east = add_prefix(rikishis, "east")

# Join with prefixed columns
main = (
    matches
    .join(west, matches.westId == west.west_id, "left")
    .join(east, matches.eastId == east.east_id, "left")
)
main.printSchema()

root
 |-- day: long (nullable = true)
 |-- division: string (nullable = true)
 |-- eastId: long (nullable = true)
 |-- eastRank: string (nullable = true)
 |-- eastShikona: string (nullable = true)
 |-- kimarite: string (nullable = true)
 |-- matchNo: long (nullable = true)
 |-- westId: long (nullable = true)
 |-- westRank: string (nullable = true)
 |-- westShikona: string (nullable = true)
 |-- winnerEn: string (nullable = true)
 |-- winnerId: long (nullable = true)
 |-- winnerJp: string (nullable = true)
 |-- match_date: date (nullable = true)
 |-- match_id: long (nullable = true)
 |-- westWin: integer (nullable = false)
 |-- west_birthDate: string (nullable = true)
 |-- west_currentRank: string (nullable = true)
 |-- west_debut: string (nullable = true)
 |-- west_height: double (nullable = true)
 |-- west_id: long (nullable = true)
 |-- west_intai: string (nullable = true)
 |-- west_weight: double (nullable = true)
 |-- west_rikishi_id: string (nullable = true)
 |-- west_yusho: long 

In [43]:
ranks = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishi_ranks/"))

In [63]:
# explode records and flatten struct fields
rows = ranks.select(explode(col("records")).alias("rank"))
flat = rows.select("rank.*")   # expand struct into columns
ranks = (flat.select("rank", "rankValue").dropDuplicates())

In [64]:
from pyspark.sql import functions as F, Window

# Filter out empty ranks
filtered = ranks.filter(F.col("rank") != "")

# Add a flag: east = 0 (better), west = 1 (worse)
with_side = filtered.withColumn(
    "side_flag",
    F.when(F.col("rank").like("%East"), F.lit(0))
     .when(F.col("rank").like("%West"), F.lit(1))
     .otherwise(F.lit(2))  # fallback if no side is present
)

# Window: order by rankValue asc, then side_flag asc
w = Window.orderBy("rankValue", "side_flag")

# Assign sequential ordering
ordered_ranks = (
    with_side
    .withColumn("order", F.row_number().over(w))
    .select("rank", "rankValue", "order")
)


In [65]:
ordered_ranks.show(20)

+-----------------+---------+-----+
|             rank|rankValue|order|
+-----------------+---------+-----+
|  Yokozuna 1 East|      101|    1|
|  Yokozuna 1 West|      101|    2|
|  Yokozuna 2 East|      102|    3|
|  Yokozuna 2 West|      102|    4|
|     Ozeki 1 East|      201|    5|
|     Ozeki 1 West|      201|    6|
|     Ozeki 2 East|      202|    7|
|     Ozeki 2 West|      202|    8|
|     Ozeki 3 East|      203|    9|
|     Ozeki 3 West|      203|   10|
|  Sekiwake 1 East|      301|   11|
|  Sekiwake 1 West|      301|   12|
|  Sekiwake 2 East|      302|   13|
|  Sekiwake 2 West|      302|   14|
|  Sekiwake 3 East|      303|   15|
|  Komusubi 1 East|      401|   16|
|  Komusubi 1 West|      401|   17|
|  Komusubi 2 East|      402|   18|
|  Komusubi 2 West|      402|   19|
|Maegashira 1 East|      501|   20|
+-----------------+---------+-----+
only showing top 20 rows


In [129]:
# Add prefixes
west = add_prefix(ordered_ranks, "west")
east = add_prefix(ordered_ranks, "east")

main = (main
       .join(west, main.westRank == west.west_rank, "left")
       .join(east, main.eastRank == east.east_rank, "left")
       )

In [130]:
# 1️⃣ Create new parsed date columns safely
main = main.withColumn(
    "east_birthdate_parsed",
    F.to_date(F.regexp_replace(F.col("east_birthdate"), "T.*Z", ""), "yyyy-MM-dd")
).withColumn(
    "west_birthdate_parsed",
    F.to_date(F.regexp_replace(F.col("west_birthdate"), "T.*Z", ""), "yyyy-MM-dd")
).withColumn(
    "match_date_parsed",
    F.to_date(F.col("match_date"), "yyyy-MM-dd")
)

# 2️⃣ Drop original ISO 8601 string columns to avoid automatic timestamp parsing
main = main.drop("east_birthdate", "west_birthdate", "match_date")

# 3️⃣ Calculate ages based on parsed dates
main = main.withColumn(
    "east_age",
    F.floor(F.months_between(F.col("match_date_parsed"), F.col("east_birthdate_parsed")) / 12)
).withColumn(
    "west_age",
    F.floor(F.months_between(F.col("match_date_parsed"), F.col("west_birthdate_parsed")) / 12)
)

# ✅ Optional: rename parsed match date to original name if needed
main = main.withColumnRenamed("match_date_parsed", "match_date")

In [131]:
# 1️⃣ Normalize debut strings: handle YYYYMM format
main = main.withColumn(
    "east_debut_clean",
    F.when(F.length("east_debut") == 6,
           F.concat_ws("-", F.col("east_debut").substr(1,4), F.col("east_debut").substr(5,2), F.lit("01"))
          ).otherwise(F.col("east_debut"))
).withColumn(
    "west_debut_clean",
    F.when(F.length("west_debut") == 6,
           F.concat_ws("-", F.col("west_debut").substr(1,4), F.col("west_debut").substr(5,2), F.lit("01"))
          ).otherwise(F.col("west_debut"))
)

# 2️⃣ Strip ISO 8601 T/Z if present and parse to date
main = main.withColumn(
    "east_debut_parsed",
    F.to_date(F.regexp_replace("east_debut_clean", "T.*Z", ""), "yyyy-MM-dd")
).withColumn(
    "west_debut_parsed",
    F.to_date(F.regexp_replace("west_debut_clean", "T.*Z", ""), "yyyy-MM-dd")
)

# 3️⃣ Calculate years active
main = main.withColumn(
    "east_years_active",
    F.floor(F.months_between("match_date", "east_debut_parsed") / 12)
).withColumn(
    "west_years_active",
    F.floor(F.months_between("match_date", "west_debut_parsed") / 12)
)

In [142]:
main = main.drop('winnerEn', 'winnerJp', 'kimarite', 'division', 'day', 'matchNo', 'match_id', 'west_intai', 'east_intai', 'west_rankValue', 'east_rankValue', 'eastRank', 'westRank', 'westId', 'eastId', 'eastShikona', 'westShikona', 'east_birthdate_parsed', 'west_birthdate_parsed', 'match_date', 'winnerId', 'west_id', 'east_id', 'west_debut', 'east_debut', 'west_debut_parsed', 'east_debut_parsed', 'east_debut_clean', 'west_debut_clean', 'west_rikishi_id', 'east_rikishi_id', 'west_currentRank', 'east_currentRank', 'west_rank', 'east_rank')

In [136]:
main.show(10)

+-------+----------------+-----------+-----------+---------------+----------+-------------------+--------------+----------------+-----------------+-----------------+----------+-------------------+-------------+--------------+---------------+----------------+-----------+-----------+---------------+----------+-------------------+--------------+----------------+-----------------+-----------------+----------+-------------------+-------------+--------------+---------------+-----------------+----------+-----------------+----------+--------+--------+-----------------+-----------------+
|westWin|west_currentRank|west_height|west_weight|west_rikishi_id|west_yusho|west_makuuchi_yusho|west_totalWins|west_totalLosses|west_totalMatches|west_makuuchiWins|west_basho|west_Makuuchi_basho|west_Gino_sho|west_Kanto_sho|west_Shukun_sho|east_currentRank|east_height|east_weight|east_rikishi_id|east_yusho|east_makuuchi_yusho|east_totalWins|east_totalLosses|east_totalMatches|east_makuuchiWins|east_basho|east_M

In [143]:
main.printSchema()

root
 |-- westWin: integer (nullable = false)
 |-- west_height: double (nullable = true)
 |-- west_weight: double (nullable = true)
 |-- west_yusho: long (nullable = true)
 |-- west_makuuchi_yusho: long (nullable = true)
 |-- west_totalWins: long (nullable = true)
 |-- west_totalLosses: long (nullable = true)
 |-- west_totalMatches: long (nullable = true)
 |-- west_makuuchiWins: long (nullable = true)
 |-- west_basho: long (nullable = true)
 |-- west_Makuuchi_basho: long (nullable = true)
 |-- west_Gino_sho: long (nullable = true)
 |-- west_Kanto_sho: long (nullable = true)
 |-- west_Shukun_sho: long (nullable = true)
 |-- east_height: double (nullable = true)
 |-- east_weight: double (nullable = true)
 |-- east_yusho: long (nullable = true)
 |-- east_makuuchi_yusho: long (nullable = true)
 |-- east_totalWins: long (nullable = true)
 |-- east_totalLosses: long (nullable = true)
 |-- east_totalMatches: long (nullable = true)
 |-- east_makuuchiWins: long (nullable = true)
 |-- east_basho

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression  # try GBTClassifier later
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

df0 = main  # your input DataFrame with the schema you posted

# 1) Drop IDs & duplicate string ranks; prefer the numeric order columns if present
cols_to_drop = ["west_rikishi_id", "east_rikishi_id", "west_rank", "east_rank", "west_currentRank", "east_currentRank"]
df1 = df0.drop(*[c for c in cols_to_drop if c in df0.columns])

# 2) Safe ratio helper (avoid division by 0)
def safe_ratio(numer, denom, default=F.lit(None)):
    return F.when(denom.isNull() | (denom == 0), default).otherwise(numer / denom)

# 3) Core win-rate features for both sides
df2 = (
    df1
    .withColumn("west_winRate", safe_ratio(F.col("west_totalWins").cast("double"), F.col("west_totalMatches").cast("double")))
    .withColumn("east_winRate", safe_ratio(F.col("east_totalWins").cast("double"), F.col("east_totalMatches").cast("double")))
    .withColumn("west_makuuchiWinRate", safe_ratio(F.col("west_makuuchiWins").cast("double"), F.col("west_Makuuchi_basho").cast("double")))
    .withColumn("east_makuuchiWinRate", safe_ratio(F.col("east_makuuchiWins").cast("double"), F.col("east_Makuuchi_basho").cast("double")))
)

# 4) Relative (west - east) deltas—these are often the most predictive
delta_pairs = [
    ("height", "double"),
    ("weight", "double"),
    ("age", "double"),
    ("years_active", "double"),
    ("winRate", "double"),
    ("makuuchiWinRate", "double"),
    ("yusho", "double"),
    ("makuuchi_yusho", "double"),
    ("totalWins", "double"),
    ("totalLosses", "double"),
    ("totalMatches", "double"),
    ("Makuuchi_basho", "double"),
    ("Gino_sho", "double"),
    ("Kanto_sho", "double"),
    ("Shukun_sho", "double"),
    ("order", "double"),  # ordinal rank (lower is better)
]

df3 = df2
for base, _ in delta_pairs:
    w = F.col(f"west_{base}").cast("double")
    e = F.col(f"east_{base}").cast("double")
    if f"west_{base}" in df2.columns and f"east_{base}" in df2.columns:
        df3 = df3.withColumn(f"d_{base}", w - e)

# 5) Choose model features
side_feats = [
    # side-specific numeric stats you want to keep in addition to deltas
    "west_order", "east_order",
    "west_basho", "east_basho",
    "west_Makuuchi_basho", "east_Makuuchi_basho",
    "west_Gino_sho", "east_Gino_sho",
    "west_Kanto_sho", "east_Kanto_sho",
    "west_Shukun_sho", "east_Shukun_sho",
    "west_winRate", "east_winRate",
    "west_makuuchiWinRate", "east_makuuchiWinRate",
    "west_height", "east_height",
    "west_weight", "east_weight",
    "west_age", "east_age",
    "west_years_active", "east_years_active",
    "west_totalMatches", "east_totalMatches",
]

delta_feats = [f"d_{b}" for (b, _) in delta_pairs if f"d_{b}" in df3.columns]

all_features = [c for c in side_feats + delta_feats if c in df3.columns]

# 6) Impute missing numeric values
imputer = Imputer(
    inputCols=all_features,
    outputCols=[c + "_imp" for c in all_features]
).setStrategy("median")

imp_features = [c + "_imp" for c in all_features]

# 7) Assemble & (for linear model) scale
assembler = VectorAssembler(inputCols=imp_features, outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withMean=True, withStd=True)

# 8) Class imbalance: weightCol
# weight = (neg_count / pos_count) for positive class; 1.0 for negative class
label_col = "westWin"
counts = df3.groupBy(label_col).count().collect()
count_map = {row[label_col]: row["count"] for row in counts}
pos = count_map.get(1, 1)
neg = count_map.get(0, 1)
pos_weight = float(neg) / float(max(pos, 1))

df4 = df3.withColumn(
    "classWeight",
    F.when(F.col(label_col) == 1, F.lit(pos_weight)).otherwise(F.lit(1.0))
)

# 9) Train/validation split (start with random; consider time-based by basho later)
train, test = df4.randomSplit([0.8, 0.2], seed=42)

# 10) Baseline model
lr = LogisticRegression(
    labelCol=label_col,
    featuresCol="features",
    weightCol="classWeight",
    maxIter=100,
    elasticNetParam=0.0,   # L2 baseline; try a grid later
    regParam=0.01
)

pipeline = Pipeline(stages=[imputer, assembler, scaler, lr])
model = pipeline.fit(train)

pred = model.transform(test)

# 11) Evaluation
auc = BinaryClassificationEvaluator(
    labelCol=label_col,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
).evaluate(pred)

aupr = BinaryClassificationEvaluator(
    labelCol=label_col,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderPR"
).evaluate(pred)

acc = MulticlassClassificationEvaluator(
    labelCol=label_col,
    predictionCol="prediction",
    metricName="accuracy"
).evaluate(pred)

print(f"AUC:  {auc:.4f}")
print(f"AUPR: {aupr:.4f}")
print(f"ACC:  {acc:.4f}")

# Optional: confusion matrix quick view
cm = (pred
      .groupBy("prediction", label_col)
      .count()
      .orderBy("prediction", label_col))
cm.show()

# 12) Save the model for later scoring
# model.write().overwrite().save("models/sumo_westwin_lr_pipeline")
